# 인수를 순회할 때는 방어적으로 하자

파라미터로 객체의 리스트를 받는 함수에서 리스트를 여러 번 순회해야할 때가 종종 있다. 

## 예를 들어 정규화 함수를 살펴보자

In [3]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [4]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


## 작업을 수행하는 generator 정의

In [9]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

generator의 반환 값에 normalize 호출 시 아무 결과도 생성되지 않는다. --> iterator가 결과를 한 번만 생성하기 때문

이미 StopIteration 예외를 일으킨 iterator나 generator를 순회해도 결과를 얻을 수 없다.

In [18]:
import random

with open('/tmp/my_numbers.txt', 'w') as f:
    for i in visits:
        f.write(str(i)+'\n')

it = read_visits('/tmp/my_numbers.txt')
percentages = normalize(it)
print(percentages)

[]


In [20]:
it = read_visits('/tmp/my_numbers.txt')
print(list(it))
print(list(it))   # 이미 소진함

[15, 35, 80]
[]


소진한 iterator를 순회하더라도 오류가 일어나지 않는다. for 루프와 list 생성자, 파이썬 표준 라이브러리의 많은 함수는 정상적인 동작 과정에서 StopIteration 예외가 일어날 것이라고 기대한다. 이런 함수는 결과가 없는 iterator 결과가 있었지만 이미 소진한 iterator의 차이를 알려주지 않는다.

이 문제를 해결하려면 입력 iterator를 명시적으로 소진하고 전체 콘텐츠의 복사본을 리스트에 저장해야 한다. 그러고 나면 리스트 버전의 데이터를 필요한 만큼 순회할 수 있다. 다음은 이전과 동일하지만 입력 iterator를 방어적으로 복사하는 함수다.

In [21]:
def normalize_copy(numbers):
    numbers = list(numbers)
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [22]:
it = read_visits('/tmp/my_numbers.txt')
percentages = normalize_copy(it)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


입력받은 iterator 콘텐츠의 복사본이 클 경우 문제가 생긴다. __이런 iterator를 복사하면 프로그램의 메모리가 고갈되어 동작을 멈출 수도 있다.__ 이를 피하기 위해 새 iterator를 반환하는 함수를 받게 만드는 것이다.

In [23]:
def normalize_func(get_iter):
    total = sum(get_iter())   # 새 iterator
    result = []
    for value in get_iter():  # 새 iterator
        percent = 100 * value / total
        result.append(percent)
    return result

normalize_func을 사용하려면 generator를 호출해서 매번 새 iterator를 생성하는 lambda 표현식을 넘겨주면 된다.

In [25]:
path = '/tmp/my_numbers.txt'
percentages = normalize_func(lambda: read_visits(path))

코드가 잘 동작하긴 하지만, 이렇게 lambda 함수를 넘겨주는 방법은 좋지 않다. 같은 결과를 얻는 더 좋은 방법은 iterator protocol을 구현한 새 컨테이너 클래스를 제공하는 것이다.

iterator protocol은 Python의 for 루프와 관련 표현식이 컨테이너 타입의 콘텐츠를 탐색하는 방법을 나타낸다.

Python은 for x in foo 같은 문장을 만나면 실제로는 iter(foo)를 호출한다. 그러면 내장 함수 iter는 `foo.__iter__` 를 호출한다. `__iter__` method는 iterator 객체를 반환해야 한다. 마지막으로 for 루프는 iterator를 모두 소진할 때 까지 iterator 객체에 내장 함수 next를 계속 호출한다.

클래스의 `__iter__` method를 generator로 구현하면 된다.

In [26]:
class ReadVisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

In [27]:
visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


### 코드 동작 이유

normalize의 sum method 가 새 iterator 객체를 할당하려고 `ReadVisits.__iter__`를 호출하기 때문이다. 숫자를 정규화하는 for 루프도 두 번째 iterator 객체를 할당할 때 `__iter__`를 호출한다. 두 iterator는 독립적으로 동작하므로 각각의 순회 과정에서 모든 입력 데이터 값을 얻을 수 있다. 이 방법의  유일한 단점은 입력 데이터를 여러 번 읽는 다는 점이다.

### 파라미터가 단순한 iterator가 아님을 보장하는 함수 작성

protocol에 따르면 __내장 함수 iter에 iterator를 넘기면 iterator 자체가 반환된다.__  
반면에 iter에 __컨테이너 타입을 넘기면 매번 새 iterator 객체가 반환된다.__

따라서 입력값을 테스트해서 iterator면 TypeError를 일으켜 거부하게 만들면 된다.

In [28]:
def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers): # iterator -- 거부!
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

### normalize_defensive는 입력 데이터를 여러 번 순회해야 할 때 사용하면 좋다.

list와 ReadVisits를 입력으로 받으면 입력이 컨테이너이므로 기대한 대로 동작한다. iterator protocol을 따르는 어떤 컨테이너 타입에 대해서도 동작한다.

In [29]:
visits = [15, 35, 80]
normalize_defensive(visits) # No Error
visits = ReadVisits(path)
normalize_defensive(visits) # No Error

[11.538461538461538, 26.923076923076923, 61.53846153846154]

함수는 입력이 iterable이어도 컨테이너가 아니면 예외를 일으킨다.

In [30]:
it = iter(visits)
normalize_defensive(it)

TypeError: Must supply a container

컨테이너 타입: list, tuple, dictionary